# Jonathan Halverson
# Wednesday, December 6, 2017
# RNN applied to MNIST

While RNN's are usually used to predict the future (since they have memory in a sense), here we use one to do image classification.

In [1]:
import tensorflow as tf

In [2]:
tf.reset_default_graph()

In [3]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

In [4]:
learning_rate = 0.001

In [5]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

In [6]:
#my_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
my_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, state_is_tuple=False)
outputs, states = tf.nn.dynamic_rnn(my_cell, X, dtype=tf.float32)

In [7]:
# use only the final state of the cell
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

In [8]:
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
init = tf.global_variables_initializer()

In [10]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('/tmp/data/')
X_test = mnist.test.images.reshape(-1, n_steps, n_inputs)
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [11]:
n_epochs = 10
batch_size = 150

In [12]:
with tf.Session() as sess:
     init.run()
     for epoch in range(n_epochs):
          for iteration in range(mnist.train.num_examples // batch_size):
               X_batch, y_batch = mnist.train.next_batch(batch_size)
               X_batch = X_batch.reshape((-1, n_steps, n_inputs))
               sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
          acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
          acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
          print(epoch, 'Train accuracy=', acc_train, "Test accuracy=", acc_test)

(0, 'Train accuracy=', 0.99333334, 'Test accuracy=', 0.94679999)
(1, 'Train accuracy=', 0.96666664, 'Test accuracy=', 0.96210003)
(2, 'Train accuracy=', 0.98666668, 'Test accuracy=', 0.96179998)
(3, 'Train accuracy=', 0.98666668, 'Test accuracy=', 0.97579998)
(4, 'Train accuracy=', 0.99333334, 'Test accuracy=', 0.97750002)
(5, 'Train accuracy=', 0.98666668, 'Test accuracy=', 0.98150003)
(6, 'Train accuracy=', 0.99333334, 'Test accuracy=', 0.98269999)
(7, 'Train accuracy=', 1.0, 'Test accuracy=', 0.98329997)
(8, 'Train accuracy=', 0.99333334, 'Test accuracy=', 0.98000002)
(9, 'Train accuracy=', 0.98666668, 'Test accuracy=', 0.97939998)


#### How does a simple linear model compare?

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

In [14]:
X_train = mnist.train.images
y_train = mnist.train.labels

In [15]:
std_sc = StandardScaler()
X_train_std = std_sc.fit_transform(X_train)
X_test_std = std_sc.transform(X_test.reshape(-1, n_steps * n_inputs))

In [16]:
SGDClassifier(tol=1e-3, alpha=0.01, max_iter=1500).fit(X_train_std, y_train).score(X_test_std, y_test)

0.90010000000000001

In [17]:
SGDClassifier(tol=1e-3, alpha=0.0001, max_iter=1500).fit(X_train_std, y_train).score(X_test_std, y_test)

0.90010000000000001

In [18]:
SGDClassifier(tol=1e-3, alpha=0.00001, max_iter=1500).fit(X_train_std, y_train).score(X_test_std, y_test)

0.89929999999999999

#### And a nonlinear model

In [19]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier(n_estimators=50).fit(X_train_std, y_train).score(X_test_std, y_test)

0.96779999999999999

We see that the RNN outperforms RF. A lot more could be done to optimize the RNN as well as preprocess the data.